## Configuration Spark

In [1]:
import socket

def check_port(host, port):
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.settimeout(5)  # Timeout de 5 secondes
    result = sock.connect_ex((host, port))
    if result == 0:
        print(f"Connexion réussie à {host}:{port}")
    else:
        print(f"Impossible de se connecter à {host}:{port}")
    sock.close()

check_port("kafka1", 9092)


Connexion réussie à kafka1:9092


In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext

conf = SparkConf() \
    .setAppName('SparkApp') \
    .setMaster('spark://spark:7077') \
    .set("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4,org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.3") \
    .set("spark.sql.shuffle.partitions", "10")
 

sc = SparkContext.getOrCreate(conf=conf)


# Créer un SQLContext pour les opérations SQL
sql_context = SQLContext(sc)

:: loading settings :: url = jar:file:/opt/conda/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-7f76866d-52c6-4eea-81d9-e55dab920b1c;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.3 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.3 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 35306)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.12/socketserver.py", line 318, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/lib/python3.12/socketserver.py", line 349, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/lib/python3.12/socketserver.py", line 362, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/lib/python3.12/socketserver.py", line 761, in __init__
    self.handle()
  File "/opt/conda/lib/python3.12/site-packages/pyspark/accumulators.py", line 295, in handle
    poll(accum_updates)
  File "/opt/conda/lib/python3.12/site-packages/pyspark/accumulators.py", line 267, in poll
    if self.rfile in r and func():
                           ^^^^^^
  File "/opt/conda/lib/python3.12/site-packages/pyspark/accumulators.p

## Imports

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, count, window, avg, to_timestamp, min, max, last, broadcast, current_timestamp
from pyspark.sql.functions import sum as spark_sum
from pyspark.sql.types import StructType, StructField, StringType, FloatType, IntegerType, DoubleType, BooleanType

## Schémas et Topics


In [4]:
kafka_broker = "kafka1:9092"
arrets_topic = "arrets"
velos_topic = "velos"
relais_topic = "relais"

# Définition du schéma pour le topic "arrets"
schema_arrets = StructType([
    StructField("codeLieu", StringType()),
    StructField("libelle", StringType()),
    StructField("distance", FloatType()),
    StructField("ligne", StringType())
])

# Définition du schéma pour le champ "position"
position_schema = StructType([
    StructField("lon", DoubleType()),
    StructField("lat", DoubleType())
])

# Définition du schéma pour le topic "velos"
schema_velos = StructType([
    StructField("number", StringType()),
    StructField("name", StringType()),
    StructField("address", StringType()),
    StructField("position", position_schema),
    StructField("banking", StringType()),
    StructField("bonus", StringType()),
    StructField("status", StringType()),
    StructField("contract_name", StringType()),
    StructField("bike_stands", IntegerType()),
    StructField("available_bike_stands", IntegerType()),
    StructField("available_bikes", IntegerType()),
    StructField("last_update", StringType())
])

# Définition du schéma pour le topic "relais"
schema_relais = StructType([
    StructField("grp_identifiant", StringType()),
    StructField("grp_nom", StringType()),
    StructField("grp_statut", IntegerType()),
    StructField("grp_disponible", IntegerType()),
    StructField("grp_exploitation", IntegerType()),
    StructField("grp_complet", IntegerType()),
    StructField("grp_horodatage", StringType()),
    StructField("idobj", StringType()),
    StructField("location", StructType([
        StructField("lon", DoubleType()),
        StructField("lat", DoubleType())
    ])),
    StructField("disponibilite", DoubleType())
])

# Définition du schéma pour le topic "relais"
schema_parking_public = StructType([
    StructField("grp_identifiant", StringType(), True),
    StructField("grp_nom", StringType(), True),
    StructField("grp_statut", StringType(), True),
    StructField("grp_disponible", IntegerType(), True),
    StructField("grp_exploitation", IntegerType(), True),
    StructField("grp_complet", IntegerType(), True),
    StructField("grp_horodatage", StringType(), True),
    StructField("idobj", StringType(), True),
    StructField("location", StructType([
         StructField("lon", DoubleType(), True),
         StructField("lat", DoubleType(), True)
    ]), True)
])

## Requête Batch sans fenêtre

Comptage des arrêts par ligne à partir du topic « arrets »

In [5]:
# Lecture brute des messages Kafka
df_kafka = sql_context.read \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_broker) \
    .option("subscribe", arrets_topic) \
    .option("startingOffsets", "earliest") \
    .load()

# Conversion du champ "value" (encodé en JSON) en colonnes structurées
df_arrets = df_kafka.selectExpr("CAST(value AS STRING) AS message") \
    .select(from_json(col("message"), schema_arrets).alias("data")) \
    .select("data.*")

# Agrégation : compter le nombre d'arrêts par "ligne"
result = df_arrets.groupBy("ligne").agg(count("*").alias("nb_arrets"))

# Affichage du résultat
result.show()
result.describe().show()

25/02/17 22:16:02 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


+--------------------+---------+
|               ligne|nb_arrets|
+--------------------+---------+
| [{"numLigne":"79"}]|        6|
|[{"numLigne":"119...|        7|
|[{"numLigne":"105...|        4|
|[{"numLigne":"127...|        2|
|[{"numLigne":"10"...|        1|
| [{"numLigne":"91"}]|       14|
|[{"numLigne":"75"...|        6|
|[{"numLigne":"50"...|        4|
|[{"numLigne":"101...|        2|
| [{"numLigne":"30"}]|       12|
| [{"numLigne":"80"}]|       10|
|[{"numLigne":"2B"...|        1|
|[{"numLigne":"112...|        1|
|[{"numLigne":"59"...|        3|
|[{"numLigne":"172...|        2|
|[{"numLigne":"30"...|        7|
|[{"numLigne":"10"...|        1|
|[{"numLigne":"115"}]|        1|
|[{"numLigne":"179...|        2|
|[{"numLigne":"107...|        3|
+--------------------+---------+
only showing top 20 rows



25/02/17 22:16:07 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


+-------+--------------------+-----------------+
|summary|               ligne|        nb_arrets|
+-------+--------------------+-----------------+
|  count|                 434|              434|
|   mean|                NULL|2.686635944700461|
| stddev|                NULL|3.224968441931372|
|    min|                  []|                1|
|    max|[{"numLigne":"NBI"}]|               28|
+-------+--------------------+-----------------+



Analyse des stations de vélo par statut historique

In [6]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql import Window

spark = SparkSession.builder.appName("BikeHistory").getOrCreate()

# --- Helper function: read_kafka_stream ---
def read_kafka_stream(topic, schema, is_streaming=True):
    if is_streaming:
        df = spark.readStream.format("kafka") \
            .option("kafka.bootstrap.servers", kafka_broker) \
            .option("subscribe", topic) \
            .option("startingOffsets", "earliest") \
            .load()
    else:
        df = spark.read.format("kafka") \
            .option("kafka.bootstrap.servers", kafka_broker) \
            .option("subscribe", topic) \
            .option("startingOffsets", "earliest") \
            .load()
    # Convert the binary "value" column to string and parse JSON using the provided schema.
    df = df.selectExpr("CAST(value AS STRING) as json_string")
    df = df.select(F.from_json(F.col("json_string"), schema).alias("data")).select("data.*")
    return df

# Read batch data from the "velos" topic (is_streaming=False)
df_velos = read_kafka_stream("velos", schema_velos, is_streaming=False).where(F.col("status").isNotNull())

# Perform aggregation by 'status'
result_batch1 = df_velos.groupBy("status") \
    .agg(
        F.count("*").alias("total_stations"),
        F.avg("available_bikes").alias("moyenne_velos_dispos"),
        F.stddev("available_bikes").alias("ecart_type_velos")
    ) \
    .withColumn("pourcentage", F.col("total_stations") / F.sum("total_stations").over(Window.partitionBy())) \
    .orderBy(F.desc("total_stations"))

result_batch1.show()


25/02/17 22:16:15 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
25/02/17 22:16:15 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
25/02/17 22:16:15 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/17 22:16:15 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/17 22:16:15 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/17 22:16:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/17 22:16:16 WARN 

+------+--------------+--------------------+------------------+-----------+
|status|total_stations|moyenne_velos_dispos|  ecart_type_velos|pourcentage|
+------+--------------+--------------------+------------------+-----------+
|  OPEN|          4606|   8.278766825879288|6.2154943408644705|       0.98|
|CLOSED|            94|                 0.0|               0.0|       0.02|
+------+--------------+--------------------+------------------+-----------+



25/02/17 22:16:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/17 22:16:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


## Requête Batch avec fenêtre

Vélos disponibles par contrat sur des fenêtres temporelles à partir du topic « velos ».
Malheuresement on a qu'une donnée par station donc ca permet pas de faire de l'aggrégation comme la moyenne de vélo sur la période..... faut essayer de trouver des données plus pertinentes.

In [7]:
# Lecture brute des messages Kafka
df_kafka = sql_context.read \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_broker) \
    .option("subscribe", velos_topic) \
    .option("startingOffsets", "earliest") \
    .load()

# Conversion du champ "value" en JSON
df_velos = df_kafka.selectExpr("CAST(value AS STRING) AS message") \
    .select(from_json(col("message"), schema_velos).alias("data")) \
    .select("data.*")

# Conversion de "last_update" en timestamp (format ISO 8601)
df_velos = df_velos.withColumn("last_update_ts", 
                               to_timestamp(col("last_update"), "yyyy-MM-dd'T'HH:mm:ssXXX"))

# Agrégation par fenêtre (ici, on allonge la fenêtre à 30 minutes pour accumuler plusieurs mises à jour) et par station
result_window = df_velos.groupBy(
    window(col("last_update_ts"), "30 minutes"), 
    col("number"),
    col("contract_name")
).agg(
    count("*").alias("nb_updates"),
    last("available_bikes", ignorenulls=True).alias("latest_available_bikes")
)

result_window.show()

25/02/17 22:16:54 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


+--------------------+------+-------------+----------+----------------------+
|              window|number|contract_name|nb_updates|latest_available_bikes|
+--------------------+------+-------------+----------+----------------------+
|{2022-05-23 14:30...|  1011|       nantes|        48|                     0|
|{2025-02-17 19:30...|    34|       nantes|        28|                     6|
|{2025-02-17 19:00...|     5|       nantes|         6|                     2|
|{2025-02-17 19:30...|    70|       nantes|        36|                    50|
|{2025-02-17 19:00...|    59|       nantes|         6|                     3|
|{2025-02-17 19:30...|    90|       nantes|        34|                     4|
|{2025-02-17 19:00...|    72|       nantes|         6|                     7|
|{2025-02-17 19:30...|    83|       nantes|        18|                     6|
|{2025-02-17 19:30...|   102|       nantes|        30|                    15|
|{2025-02-17 19:30...|   103|       nantes|        28|          

## Requête Batch avec visualisation

Graphique des totaux de stations par statut avec Pandas et Seaborn

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Convert the Spark DataFrame to Pandas for visualization
pdf = result_batch1.toPandas()

plt.figure(figsize=(10,6))
sns.barplot(data=pdf, x="status", y="total_stations", palette="viridis")
plt.title("Total des stations de vélo par statut")
plt.xlabel("Statut")
plt.ylabel("Nombre total de stations")
plt.show()


## Requêtes Streaming

### Récuperer la moyenne des disponiblités des parkings relais sur une fenêtre glissante.

In [ ]:
# Lecture en streaming depuis Kafka
df_kafka = sql_context.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_broker) \
    .option("subscribe", relais_topic) \
    .option("startingOffsets", "latest") \
    .load()

# Conversion du champ "value" (JSON) en colonnes structurées
df_relais = df_kafka.selectExpr("CAST(value AS STRING) AS message") \
    .select(from_json(col("message"), schema_relais).alias("data")) \
    .select("data.*")

# Conversion du champ "grp_horodatage" en timestamp
# Le format ISO 8601 est utilisé : "yyyy-MM-dd'T'HH:mm:ssXXX"
df_relais = df_relais.withColumn("horodatage_ts", 
                                  to_timestamp(col("grp_horodatage"), "yyyy-MM-dd'T'HH:mm:ssXXX"))

# Définition d'un watermark de 5 minutes pour gérer les retards éventuels
df_relais = df_relais.withWatermark("horodatage_ts", "5 minutes")

# Agrégation : calcul de la moyenne des disponibilités sur une fenêtre glissante
# Ici, la fenêtre est de 5 minutes et on agrège par "grp_identifiant"
result_stream = df_relais.groupBy(
    window(col("horodatage_ts"), "5 minutes"),
    col("grp_identifiant"),
).agg(
    avg("disponibilite").alias("avg_disponibilite")
)

# Écriture du résultat en streaming vers la console (mode "update")
query = result_stream.writeStream \
    .outputMode("update") \
    .format("console") \
    .option("truncate", "false") \
    .start()

query.awaitTermination()

In [ ]:
query.stop()

### Récuperer la moyenne des vélos disponibles par parking sur une fenêtre glissante.

In [ ]:
# Lecture en streaming depuis Kafka
df_kafka_velos = sql_context.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_broker) \
    .option("subscribe", velos_topic) \
    .option("startingOffsets", "latest") \
    .load()

# Conversion du champ "value" (JSON) en colonnes structurées
df_velos = df_kafka_velos.selectExpr("CAST(value AS STRING) AS message") \
    .select(from_json(col("message"), schema_velos).alias("data")) \
    .select("data.*")

# Conversion du champ "last_update" en timestamp (format ISO 8601)
df_velos = df_velos.withColumn("last_update_ts", 
                               to_timestamp(col("last_update"), "yyyy-MM-dd'T'HH:mm:ssXXX"))

# Application d'un watermark de 5 minutes pour gérer les retards
df_velos = df_velos.withWatermark("last_update_ts", "5 minutes")

# Agrégation sur une fenêtre de 5 minutes : calcul du total des places disponibles
result_velos = df_velos.groupBy(
    window(col("last_update_ts"), "5 minutes"),
    col("name")
).agg(
    avg("available_bikes").alias("moyenne_velos_disponibles")
)

# Écriture du résultat en streaming vers la console (mode "update")
query_velos = result_velos.writeStream \
    .outputMode("update") \
    .format("console") \
    .option("truncate", "false") \
    .start()

query_velos.awaitTermination()

In [ ]:
query_velos.stop()

## Multimodal query

Section 1: Batch Query – Aggregated Multimodal Connectivity Hubs

In [16]:
# Batch Request: Bike Availability Near Full Public Parkings (Optimized)
# ======================================================================

# 1. Read and Validate Data
# -------------------------
# Read parking data with correct topic name
df_parking = read_kafka_stream(
    "parking_public",  # Match actual topic name
    schema_parking_public, 
    is_streaming=False
).filter(
    (F.col("location.lon").isNotNull()) &
    (F.col("location.lat").isNotNull()) &
    (F.col("grp_complet") == 1)  # Directly filter full parkings
).alias("parking")

# Read bike data with quality checks
df_velos = read_kafka_stream(
    velos_topic, 
    schema_velos, 
    is_streaming=False
).filter(
    (F.col("position.lon").isNotNull()) &
    (F.col("position.lat").isNotNull()) &
    (F.col("available_bikes") > 0)
).alias("bike")

# 2. Spatial Pre-filtering
# ------------------------
# Approximate bounding box filter (0.5° ~55km at equator)
parking = df_parking.withColumn("parking_geo", F.struct("location.lon", "location.lat"))
bikes = df_velos.withColumn("bike_geo", F.struct("position.lon", "position.lat"))

# 3. Optimized Spatial Join
# -------------------------
join_condition = [
    F.abs(parking["location.lon"] - bikes["position.lon"]) <= 0.5,
    F.abs(parking["location.lat"] - bikes["position.lat"]) <= 0.5
]

joined_df = parking.join(
    broadcast(bikes),
    join_condition,
    "inner"
)

# 4. Precise Distance Calculation
# -------------------------------
joined_df = joined_df.withColumn(
    "distance_km",
    haversine_udf(
        F.col("location.lon"), 
        F.col("location.lat"),
        F.col("position.lon"),
        F.col("position.lat")
    )
).filter(F.col("distance_km") <= 0.5)

# 5. Aggregation with Monitoring
# ------------------------------
if joined_df.rdd.isEmpty():
    print("Warning: No matching bike stations found near full parkings")
else:
    parking_bike_availability = (joined_df
        .groupBy("grp_identifiant", "grp_nom")
        .agg(
            F.sum("available_bikes").alias("total_bikes"),
            F.count_distinct("number").alias("distinct_stations")
        )
        .withColumn("status", 
            F.when(F.col("total_bikes") >= 20, "Sufficient")
            .otherwise("Insufficient"))
        .orderBy(F.desc("total_bikes"))
    )

    # Show execution plan
    parking_bike_availability.explain()
    
    # Show results
    parking_bike_availability.show(truncate=False)

25/02/17 22:29:59 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
25/02/17 22:29:59 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
25/02/17 22:30:00 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


In [22]:
# Batch Request: Deduplicated Bike Availability Analysis
# ======================================================

# 1. Window Function with Row Number
window_spec = Window.partitionBy("grp_identifiant").orderBy(
    "distance_km",
    F.desc("bike.available_bikes")  # Tiebreaker: prefer stations with more bikes
)

closest_stations = (joined_df
    .withColumn("closest_rank", F.row_number().over(window_spec))
    .filter(F.col("closest_rank") == 1)
    .select(
        "grp_identifiant",
        "grp_nom",
        F.col("parking.grp_disponible").alias("available_parking_spaces"),
        F.col("bike.number").alias("closest_station_id"),
        F.round("distance_km", 2).alias("closest_station_km")
    )
    .distinct()  # Add distinct to remove potential duplicates
)

# 2. Aggregation with Deduplication
nearby_counts = (joined_df
    .groupBy("grp_identifiant")
    .agg(
        F.count_distinct("bike.number").alias("nearby_stations"),
        F.sum("bike.available_bikes").alias("total_bikes")
    )
)

# 3. Final Join with Deduplication Check
final_result = (closest_stations
    .join(nearby_counts, "grp_identifiant")
    .withColumn("bike_availability_status",
                F.when(F.col("total_bikes") >= 15, "Good")
                 .when(F.col("total_bikes") >= 5, "Moderate")
                 .otherwise("Low"))
    .select(
        "grp_identifiant",
        "grp_nom",
        "available_parking_spaces",
        "total_bikes",
        "nearby_stations",
        "closest_station_km",
        "closest_station_id",
        "bike_availability_status"
    )
    .dropDuplicates(["grp_identifiant"])  # Ensure final deduplication
    .orderBy(F.desc("total_bikes"))
)

final_result.show(truncate=False)

25/02/17 22:51:10 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
25/02/17 22:51:10 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
25/02/17 22:51:10 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
25/02/17 22:51:10 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
25/02/17 22:51:14 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
25/02/17 22:51:

+---------------+----------------------+------------------------+-----------+---------------+------------------+------------------+------------------------+
|grp_identifiant|grp_nom               |available_parking_spaces|total_bikes|nearby_stations|closest_station_km|closest_station_id|bike_availability_status|
+---------------+----------------------+------------------------+-----------+---------------+------------------+------------------+------------------------+
|003            |Tour Bretagne         |440                     |243175     |19             |0.1               |6                 |Good                    |
|005            |Aristide Briand       |200                     |232850     |14             |0.01              |15                |Good                    |
|008            |Talensac              |0                       |225925     |15             |0.06              |19                |Good                    |
|035            |Baco-LU 1             |63                

In [26]:
# Check record counts at each stage
print("Parking Count:", df_parking.count())  # Should be >0
print("Velos Count:", df_velos.count())      # Should be >0
print("Relais Count:", df_relais.count())    # Should be >0

# Check non-full parkings
non_full = df_parking.filter(F.col("grp_complet") != 1)
print("Non-Full Parkings:", non_full.count())

25/02/17 22:56:34 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
25/02/17 22:56:35 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


Parking Count: 675
Velos Count: 6229


25/02/17 22:56:36 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
25/02/17 22:56:36 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


Relais Count: 7000
Non-Full Parkings: 675


In [38]:
# Batch Request: Average Bike Availability Near Available Parkings
parking_bike_analysis = (
    df_parking.alias("parking")
    .filter(F.col("parking.grp_complet") != 1)
    .join(
        df_velos.alias("velo"),
        [  # Properly balanced parentheses
            (F.abs(F.col("parking.location.lon") - F.col("velo.position.lon")) <= 0.01),
            (F.abs(F.col("parking.location.lat") - F.col("velo.position.lat")) <= 0.01)
        ]
    )
    .withColumn("distance_km", haversine_udf(
        F.col("parking.location.lon"), 
        F.col("parking.location.lat"),
        F.col("velo.position.lon"),
        F.col("velo.position.lat")
    ))
    .filter(F.col("distance_km") <= 0.5)
    .groupBy("parking.grp_identifiant")
    .agg(
        F.avg("velo.available_bikes").alias("avg_bikes"),
        F.count_distinct("velo.number").alias("nearby_stations"),
        F.first("parking.location.lon").alias("parking_lon"),
        F.first("parking.location.lat").alias("parking_lat")
    )
    .join(df_relais.alias("relais"), "grp_identifiant", "left")
    .select(
        F.coalesce("relais.grp_nom", "grp_identifiant").alias("location_name"),
        "avg_bikes",
        "nearby_stations",
        F.coalesce("relais.location.lon", "parking_lon").alias("lon"),
        F.coalesce("relais.location.lat", "parking_lat").alias("lat")
    )
)

parking_bike_analysis.show(truncate=False)

25/02/17 23:20:01 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
25/02/17 23:20:01 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
25/02/17 23:20:01 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


+-------------+------------------+---------------+-------------------+------------------+
|location_name|avg_bikes         |nearby_stations|lon                |lat               |
+-------------+------------------+---------------+-------------------+------------------+
|004          |10.725738396624472|13             |-1.562571814000023 |47.214310205000004|
|015          |5.321266968325792 |9              |-1.5519831099999806|47.21105707999999 |
|017          |4.243161094224924 |8              |-1.5572502320000012|47.20936196600002 |
|047          |6.5886075949367084|4              |-1.5632873819999986|47.20409254200001 |
|006          |7.580645161290323 |10             |-1.56233817399999  |47.210870821000015|
|009          |12.994525547445255|9              |-1.5439712740000004|47.212901665000004|
|001          |6.221142162818955 |15             |-1.552558781000016 |47.21407529499999 |
|042          |5.213793103448276 |7              |-1.55291388400002  |47.20889024799999 |
|035      